In [185]:
library(bsts);
library(tidyverse);
library(rsample);
library(recipes);
library(tibble);
library(glue);
library(plyr);
library(RcppCNPy);

library(timetk);
library(tidyquant);
library(tibbletime);
library(reshape2);

# Visualization
library(cowplot);

In [186]:
columns_trn <- as.vector(unlist(read.csv('../../data/metadata/headers.csv', header = F)))
columns_tst <- as.vector(unlist(read.csv('../../data/metadata/headers_anomaly.csv', header = F)))
data_trn <- as.data.frame(npyLoad('../../data/scaled/data_scaled_train.npy'))
data_tst <- read.csv('../../data_scaled_anomaly_test.csv', sep = ',')
colnames(data_trn) <- columns_trn
colnames(data_tst) <- columns_tst

In [187]:
head(data_tst)

VAL_23-FT-92537-01:X.Value,VAL_23-TT-92539:X.Value,VAL_23-PT-92539:X.Value,VAL_23-FT-92512:X.Value,VAL_23-FT-92521:X.Value,VAL_23-KA-9101-M01-62B:X.Value,VAL_23-KA-9101_APE:VALUE,VAL_23-KA-9101_APH:VALUE,VAL_23-KA-9101_APR:VALUE,VAL_23-KA-9101_ASP:VALUE,⋯,VAL_23-TT-92533:X.Value,VAL_23-VA-9110-M01-EL:XS.MeasuredValues.CurrentL2.Value.Value,VAL_23_FIC_92543_01:Z.X.Value,VAL_23_FIC_92543_04:Z.X.Value,VAL_23_FIC_92543_06:Z.X.Value,VAL_23_FIC_92543_08:Z.X.Value,VAL_23_KA_9101_M01_62C:Z.X.Value,VAL_23_PT_92531:Z.X.Value,VAL_23_TT_92532:Z.X.Value,VAL_23_ZT_92543:Z.X.Value
-0.3658386,0.3813640,0.9285194,0.76168879,-1.7537263,-0.3835543,-0.9008834,-0.230720795,-0.42092460,-0.6272200,⋯,0.27699014,-1.6619945,-0.8629771,-0.8478631,1.3381178,0.9311267,-0.4060731,1.3416014,0.25314041,-0.8692840
-1.0135175,0.5234370,0.4577381,0.50386028,-0.3276456,-1.0167395,-0.5350428,0.201365299,0.06057403,-1.0226560,⋯,0.35315446,-1.4983057,-0.7955434,-1.2207535,0.4263343,0.4516105,-1.0896832,0.4107829,0.33062404,-0.7673661
-1.6579560,0.6440973,-0.5124640,0.05413583,-0.8134358,-1.8583637,-0.1935167,0.637410173,0.54800350,-2.1325107,⋯,0.32240948,-1.0314597,-0.3406772,-1.4471922,-0.7567967,-0.5138778,-1.8545669,-0.7885011,0.35261702,-0.4431667
-1.0939451,0.6013910,-1.7245236,-0.54003362,1.0934136,-1.5318871,-1.2951613,0.002277485,-0.27160482,-0.4921659,⋯,0.15154388,-1.4453027,0.3936263,-0.1303004,-1.4998491,-1.7292265,-1.5869855,-1.4797270,0.26077215,0.3719042
1.0199087,0.2680502,1.3217167,0.65835575,-1.5930887,0.8880654,-3.0041100,-1.572278924,-2.01689039,1.9588335,⋯,-0.07290009,-1.2133490,-0.3226269,0.3820666,2.6241189,1.3258853,0.9219742,2.6792449,0.03581646,-0.1794360
0.3702588,0.1161355,2.1406182,0.66965060,-0.4340130,0.4752938,-2.0602118,-1.174059611,-1.42985140,0.7345933,⋯,0.01371629,-0.9326325,-0.5565307,-0.7651568,3.0487713,2.1375751,0.3695946,3.0518975,-0.02919511,-0.5800781


In [188]:
ncol(data_trn)
ncol(data_tst)

[1] 44

[1] 44

In [189]:
head(data_trn)

VAL_23-FT-92537-01:X.Value,VAL_23-TT-92539:X.Value,VAL_23-PT-92539:X.Value,VAL_23-FT-92512:X.Value,VAL_23-FT-92521:X.Value,VAL_23-KA-9101-M01-62B:X.Value,VAL_23-KA-9101_APE:VALUE,VAL_23-KA-9101_APH:VALUE,VAL_23-KA-9101_APR:VALUE,VAL_23-KA-9101_ASP:VALUE,⋯,VAL_23-TT-92533:X.Value,VAL_23-VA-9110-M01-EL:XS.MeasuredValues.CurrentL2.Value.Value,VAL_23_FIC_92543_01:Z.X.Value,VAL_23_FIC_92543_04:Z.X.Value,VAL_23_FIC_92543_06:Z.X.Value,VAL_23_FIC_92543_08:Z.X.Value,VAL_23_KA_9101_M01_62C:Z.X.Value,VAL_23_PT_92531:Z.X.Value,VAL_23_TT_92532:Z.X.Value,VAL_23_ZT_92543:Z.X.Value
1.10868160,-0.9017887,0.59226103,-0.7312962,0.6044069,0.84891703,-0.19909076,-0.23724863,-0.2032456,0.3808100,⋯,-0.8067785,-0.9643457,0.9085269,0.4870149,1.0100227,0.58420857,0.83487401,1.02064337,-0.8446010,1.05845706
1.67167807,-0.9846032,1.30837840,0.5894533,0.5082888,1.60264515,-0.07985224,-0.17498050,-0.1081445,0.8276711,⋯,-0.7623771,-0.9261755,-0.1505724,0.5119046,1.4373029,1.33599908,1.62887122,1.40062801,-0.8419119,0.08930794
-0.38449393,-0.7414123,0.29999474,-0.4312056,0.2842272,-0.06638881,0.48027855,0.42091143,0.6181163,-0.4368274,⋯,-0.5012369,-0.8399260,0.5107766,-0.6635086,-0.3897928,0.31555927,-0.09097357,-0.39448045,-0.6730263,0.43381135
0.01214076,-0.4747175,-0.32209754,-0.8871136,0.2089395,-0.09620551,0.03196365,0.07512782,0.1327661,0.3112018,⋯,-0.4894951,-0.8401622,0.9817740,0.1543406,-0.3515722,-0.29900505,-0.06740096,-0.34936406,-0.5470099,0.95232435
-0.85411698,-0.5616603,-0.07544699,-0.8738648,0.2928082,-0.84695068,0.03628058,0.06561971,0.1254095,0.0554855,⋯,-0.5936397,-0.8665833,0.7478092,-1.0764850,-0.4025539,-0.05317293,-0.71532143,-0.44031397,-0.5714835,0.63364078
0.93289805,-0.5862933,-0.31996769,-0.5089502,0.4264333,0.59072261,-0.24797358,-0.19311051,-0.1982187,0.3722106,⋯,-0.7276962,-0.8726398,0.7168385,0.9581853,-0.1106070,-0.31960006,0.53482685,-0.07107917,-0.6574773,1.00070277


In [190]:
lag <- 0
response <- 'VAL_23-FT-92537-01:X.Value'
names(data_trn)[names(data_trn) == response] <- 'response'
names(data_tst)[names(data_tst) == response] <- 'response'


data_tst$response <- lead(data_tst$response, lag)
data_trn$response <- lead(data_trn$response, lag)

data_tst <- na.omit(data_tst)
data_trn <- na.omit(data_trn)

x_test <- data_tst %>% select(-response)
y_test <- data_tst$response

x_train <- data_trn %>% select(-response)
y_train <- data_trn$response

In [191]:
ss <- AddLocalLevel(list(), data_trn$response)
model <- bsts(
    response ~., 
    state.specification = ss, 
    niter = 500,
    ping = 0,
    data = data_trn
)

pred_test <- predict(model, newdata = x_test)$mean
pred_train <- predict(model, newdata = x_train)$mean

In [192]:
mean(abs(y_train - pred_train))

[1] 0.07130075

In [193]:
mean(abs(y_test - pred_test))

[1] 0.2008004

In [194]:
df_train <- data.frame(y_train, pred_train)
names(df_train) <- c('y_train', 'predicted_train')

df_test <- data.frame(y_test, pred_test)
names(df_test) <- c('y_test', 'predicted_test')

In [195]:
write.table(
    df_train, 
    "../../data/bsts_prediction_train.csv",
    row.names = FALSE,
    sep = ","
)

write.table(
    df_test, 
    "../../data/bsts_prediction_test.csv",
    row.names = FALSE,
    sep = ","
)